In [51]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time
import numpy as np
import rotation_matrix as rm
import pandas as pd
import sys
import os
import warnings

# add parent folder to sys.path
sys.path.insert(0, os.path.abspath(".."))

# ignore DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ignore RuntimeWarning messages
warnings.filterwarnings("ignore", category=RuntimeWarning)

from volume_estimation.src.models_1_no_vol.predict_full_pipeline import predict_no_vol
from segmentation_and_depth.src.models.predict_model import predict as predict_seg_depth
from volume_estimation.src.models_2_input_vol.predict_full_pipeline import predict_with_vol

ModuleNotFoundError: No module named 'torch'

In [2]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

rtde_c, rtde_r, rtde_io_set = connect_robot()

In [9]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(10)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")
gripper.move(45)

Activating Gripper
Gripper activated


True

In [15]:
# open csv file
test_bottles = pd.read_csv('../pouring_simulation/output/summary_medium.csv')

In [16]:
#print first 50 rows
test_bottles[0:50]

,scene_number,path,rotationSpeed,stop_angle,pause_time,volume_start,volume_poured,volume_received,spilled_volume
0,1,../../output/MediumBottle/Medium_50_200_20,0.03,20.0,0.2,0.000,0.000000,0.000000,0.0000
1,2,../../output/MediumBottle/Medium_50_200_22,0.03,22.0,0.2,49.995,0.000000,0.000000,0.0000
2,3,../../output/MediumBottle/Medium_50_200_24,0.03,24.0,0.2,49.995,0.002500,0.002500,0.0000
3,4,../../output/MediumBottle/Medium_50_200_26,0.03,26.0,0.2,49.995,0.000000,0.000000,0.0000
4,5,../../output/MediumBottle/Medium_50_200_28,0.03,28.0,0.2,49.995,0.002500,0.000000,0.0025
5,6,../../output/MediumBottle/Medium_50_200_30,0.03,30.0,0.2,49.995,0.005000,0.005000,0.0000
6,7,../../output/MediumBottle/Medium_50_200_32,0.03,32.0,0.2,49.995,0.002500,0.002500,0.0000
7,8,../../output/MediumBottle/Medium_50_200_34,0.03,34.0,0.2,49.995,0.000000,0.000000,0.0000
8,9,../../output/MediumBottle/Medium_50_200_36,0.03,36.0,0.2,49.995,0.000000,0.000000,0.0000
9,10,../../output/MediumBottle/Medium_50_200_38,0.03,38.0,0.2,49.995,0.000000,0.000000,0.0000


## Manually move robot to start position and read the position

In [8]:
start_pos_horizontal = rtde_r.getActualTCPPose()

In [8]:
start_pos_horizontal_old = [-0.05483472191563013+0.001,
 0.5215426516767572,
 0.16428408481957893+0.005,
 -1.5709891478285878,
 0.035603245728593275,
 0.03565316928565675]

In [6]:
start_pos_horizontal = rtde_r.getActualTCPPose()

In [9]:
start_pos_horizontal

[-0.36498883237255164,
 0.24715099595577825,
 0.17519255890890678,
 -1.221741554832158,
 -1.213214812915213,
 1.2457124592502689]

In [32]:
start_pos_horizontal = [-0.36498883237255164,
 0.24715099595577825+0.003,
 0.17519255890890678,
 -1.221741554832158,
 -1.213214812915213,
 1.2457124592502689]

In [13]:
j_start_joints = rtde_r.getActualQ()

In [14]:
j_start_joints = [-3.7063358465777796,
 -1.5917755566039027,
 -2.5010504722595215,
 4.07737843572583,
 -2.1236422697650355,
 -3.1435285250293177]

[-3.7063358465777796,
 -1.5917755566039027,
 -2.5010504722595215,
 4.07737843572583,
 -2.1236422697650355,
 -3.1435285250293177]

In [33]:
rtde_c.moveL(start_pos_horizontal, 0.1, 0.1)

True

In [34]:
init_q = rtde_r.getActualQ()
init_q[5] += 0.25307274
rtde_c.moveJ(init_q, 0.1, 0.1)

start_pos_bottle_rotated = rtde_r.getActualTCPPose()

In [36]:
start_pos_bottle_rotated

[-0.36498143802471317,
 0.2501417037919356,
 0.17515693872394192,
 -1.4078543077400933,
 -1.0821978511944732,
 1.4284556202410212]

In [10]:
gripper.move(45)

True

### Move to picking up position

In [14]:
pos = rtde_r.getActualTCPPose()

In [15]:
pos

[-0.3979018097425171,
 0.1980390973153733,
 0.28823304066468763,
 -1.3004855590335682,
 -1.1848976095494719,
 1.213794766836109]

In [38]:
pick_up_pos_1 = [-0.20933143110289607,
 0.4007959363962377,
 0.14113332170567885,
 -1.559202807626822,
 0.03734946834843037,
 0.004841554724431051]

pick_up_pos_2 = [-0.23954717580833165,
 0.400795552537626,
 0.07130884774090128,
 -1.5591976930082265,
 0.037387010510812266,
 0.004880320202878557]

pick_up_pos_4 = [-0.23473853377504333,
 0.47152953670488773,
 0.053494416798714306,
 -1.5710858755788153,
 0.017660070482072698,
 0.01776629011841677] # gripping_pos

move_to_pouring_pos_1 = [-0.23473853377504333,
 0.47152953670488773,
 0.183494416798714306,
 -1.5710858755788153,
 0.017660070482072698,
 0.01776629011841677]

move_to_pouring_pos_2 = [-0.22901634283841804,
 0.41998903523904,
 0.2949162372788959,
 -1.5751694238784661,
 -0.21652305912711,
 0.25229985538985844]

move_to_pouring_pos_3 = [-0.3979018097425171,
 0.1980390973153733,
 0.28823304066468763,
 -1.3004855590335682,
 -1.1848976095494719,
 1.213794766836109]

start_pos_horizontal = [-0.36498883237255164,
 0.24715099595577825+0.003,
 0.17519255890890678,
 -1.221741554832158,
 -1.213214812915213,
 1.2457124592502689]

start_pos_bottle_rotated = [-0.36498143802471317,
 0.2501417037919356,
 0.17515693872394192,
 -1.4078543077400933,
 -1.0821978511944732,
 1.4284556202410212]

In [52]:
def get_bottle():
    rtde_c.moveL(pick_up_pos_1, 0.3, 0.3)
    rtde_c.moveL(pick_up_pos_2, 0.3, 0.3)
    gripper.open()
    rtde_c.moveL(pick_up_pos_4, 0.3, 0.3)
    gripper.close()
    rtde_c.moveL(move_to_pouring_pos_1, 0.3, 0.3)
    rtde_c.moveL(move_to_pouring_pos_2, 0.3, 0.3)
    rtde_c.moveL(move_to_pouring_pos_3, 0.3, 0.3)
    rtde_c.moveL(start_pos_bottle_rotated, 0.3, 0.3)

def place_back():
    rtde_c.moveL(start_pos_bottle_rotated, 0.3, 0.3)
    rtde_c.moveL(move_to_pouring_pos_3, 0.3, 0.3)
    rtde_c.moveL(move_to_pouring_pos_2, 0.3, 0.3)
    rtde_c.moveL(move_to_pouring_pos_1, 0.3, 0.3)
    rtde_c.moveL(pick_up_pos_4, 0.3, 0.3)
    gripper.open()
    rtde_c.moveL(pick_up_pos_2, 0.3, 0.3)
    rtde_c.moveL(pick_up_pos_1, 0.3, 0.3)

# Start here:

## Input sample number:

In [17]:
sample_number = 300

### Loading TCP file for movement

In [41]:
scene_path = test_bottles.iloc[sample_number]['path']
# delete first five characters of the path
scene_path = scene_path[5:]
scene_path = "../pouring_simulation/" + scene_path + "/TCP.txt"
#data_points = np.loadtxt(scene_path, delimiter=',', skiprows=1) # skiprows 1 and 1614

# Count the total number of rows in the file
with open(scene_path, 'r') as file:
    num_rows = sum(1 for _ in file)
# load datapoints and skip first and last 5
data_points = np.loadtxt(scene_path, delimiter=',', skiprows=1, max_rows=num_rows-2)

# convert from inches to meters
data_points[:, 0] = data_points[:, 0] * 0.0254
data_points[:, 1] = data_points[:, 1] * 0.0254
data_points[:, 2] = data_points[:, 2] + 0.25307274

# create list of positions
positions = []
for i in range(data_points.shape[0]):
    #positions.append([-data_points[i,1], -data_points[i,0], 0.0, 0.0, 0.0, data_points[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups
    positions.append([data_points[i,0], -data_points[i,1], 0.0, 0.0, 0.0, data_points[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

positions_converted = []
for i in range(data_points.shape[0]):
    # if none of the data entries is 0
    if not (positions[i][0] == 0 or positions[i][1] == 0 or positions[i][5] == 0):
        positions_converted.append(rm.PoseTrans(start_pos_horizontal, positions[i])) # transform from tool coordinate system to base coordinate system
        

### Calculate speed


In [42]:
# get first 5 x and z positions 
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*150)
    #speed.append(distances[i]*89)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

avg_speed = 0.007386997722436739

[3.873303419048123e-05, 3.874244672401903e-05, 3.876746275141092e-05, 3.8756714011425966e-05, 3.878904928949199e-05, 3.8791142625309426e-05, 3.880077789867648e-05, 3.881287767178744e-05, 3.881220961402546e-05, 3.884991931559448e-05, 3.884671255048649e-05, 3.88486574643645e-05, 3.887106062829475e-05, 3.8870466830789796e-05]
[0.005809955128572185, 0.005811367008602855, 0.005815119412711638, 0.005813507101713895, 0.0058183573934237984, 0.005818671393796414, 0.005820116684801472, 0.005821931650768116, 0.005821831442103818, 0.005827487897339173, 0.005827006882572974, 0.005827298619654675, 0.0058306590942442124, 0.00583057002461847]
Speed in m/s:  0.005820991409637406


### Calculate pause time and find pause point

In [43]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif positions_converted[i] == positions_converted[i-1] == positions_converted[i-2] == positions_converted[i-3] == positions_converted[i-4]:
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

1538
80


### Show pouring settings from simulation

In [44]:
print("Start Volume: ", test_bottles.iloc[sample_number]['volume_start'], "mL")
print("Predicted Poured Volume: ", test_bottles.iloc[sample_number]['volume_poured'], "mL")
print("Predicted Remaining Volume: ", float(test_bottles.iloc[sample_number]['volume_start'])-float(test_bottles.iloc[sample_number]['volume_poured']), "mL")
print("Predicted Received Volume: ", test_bottles.iloc[sample_number]['volume_received'], "mL")
print("Predicted Spilled Volume: ", test_bottles.iloc[sample_number]['spilled_volume'], "mL")
print("\n")
print("Stop Angle: ", test_bottles.iloc[sample_number]['stop_angle'], "degrees")
print("Stop Time: ", test_bottles.iloc[sample_number]['pause_time'], "seconds")

Start Volume:  59.99 mL
Predicted Poured Volume:  0.0025 mL
Predicted Remaining Volume:  59.987500000000004 mL
Predicted Received Volume:  0.0 mL
Predicted Spilled Volume:  0.0025 mL


Stop Angle:  44.0 degrees
Stop Time:  1.4 seconds


### Do pouring

In [53]:
time.sleep(2)

get_bottle()

velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.0015
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(len(positions_converted1)-1):
    path.append([positions_converted1[i][0], positions_converted1[i][1], positions_converted1[i][2], positions_converted1[i][3], positions_converted1[i][4], positions_converted1[i][5], velocity, acceleration, blend_i])

path.append([positions_converted1[-1][0], positions_converted1[-1][1], positions_converted1[-1][2], positions_converted1[-1][3], positions_converted1[-1][4], positions_converted1[-1][5], velocity, acceleration, 0])
rtde_c.moveL(path)

time.sleep(count/150)
#time.sleep(count/60)

path_2 = []
for i in range(len(positions_converted2)-1):
    path_2.append([positions_converted2[i][0], positions_converted2[i][1], positions_converted2[i][2], positions_converted2[i][3], positions_converted2[i][4], positions_converted2[i][5], velocity, acceleration, blend_i])

path_2.append([positions_converted2[-1][0], positions_converted2[-1][1], positions_converted2[-1][2], positions_converted2[-1][3], positions_converted2[-1][4], positions_converted2[-1][5], velocity, acceleration, blend_3])

rtde_c.moveL(path_2)

#rtde_c.stopScript()

time.sleep(1)
gripper.close()



True

In [54]:
place_back()

### Measure results, write to excel file and restart